In [0]:
import nltk
import os
import string
import numpy as np
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from pathlib import Path

In [2]:
!pip install num2words

In [0]:
import num2words
from num2words import num2words

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = Path('/content/drive/My Drive/assignment_1_mod')

In [6]:
cd /content/drive/My Drive/assignment_1_mod

/content/drive/My Drive/assignment_1_mod


In [7]:
dataset = os.listdir(path)
print(dataset)

['d1.txt', 'd3.txt', 'd2.txt', 'd4.txt', 'd5.txt']


In [0]:
def print_document(id):
    print(dataset[id])
    file = open(dataset[id], 'r')
    text = file.read().strip()
    file.close()
    print(text)

<h2>Preprocessing of dataset</h2>

In [0]:
def lowercase(inputs):
    return np.char.lower(inputs)

In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def stopwords_remover(inputs):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(inputs))
    modified_text = ""
    for word in words:
        if word not in stop_words and len(word) > 1:
            modified_text = modified_text + " " + word
    return modified_text

In [0]:
def punctuation_remover(inputs):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        inputs = np.char.replace(inputs, symbols[i], ' ')
        inputs = np.char.replace(inputs, "  ", " ")
    inputs = np.char.replace(inputs, ',', '')
    inputs = np.char.replace(inputs, "'", "")
    return inputs   

In [0]:
def stemming(inputs):
    stemmer = PorterStemmer()
    tokens = word_tokenize(str(inputs))
    modified_text = ""
    for word in tokens:
        modified_text = modified_text + " " + stemmer.stem(word)
    return modified_text

In [0]:
def convert_numbers(inputs):
    tokens = word_tokenize(str(inputs))
    modified_text = ""
    for word in tokens:
        try:
            word = num2words(int(word))
        except:
            a = 0
        modified_text = modified_text + " " + word
    modified_text = np.char.replace(modified_text, "-", " ")
    return modified_text

In [0]:
def preprocess(text):
    text = lowercase(text)
    text = punctuation_remover(text) 
    text = stopwords_remover(text)
    text = convert_numbers(text)
    text = stemming(text)
    text = punctuation_remover(text)
    text = convert_numbers(text)
    text = stemming(text) 
    text = punctuation_remover(text) 
    text = stopwords_remover(text)
    return text

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
processed_text = []
for i in dataset:
    file = open(i, 'r', errors='ignore')
    text = file.read()
    file.close()
    processed_text.append(word_tokenize(str(preprocess(text))))

In [0]:
processed_text

In [0]:
N = len(dataset)
# N = 5

'processed_text' contains a list of lists of the words occurring in each document

Creating document frequency(nj in idf)

In [0]:
nj = {} 
# nj is a dictionary that is of the format key : {}
# key is the word from a list and {} is a set containing all the documents in which 'key' is occuring
for i in range(N):
    tokens = processed_text[i]
    for word in tokens:
        try:
          # when the set contains some element in it add a new element to the set
            nj[word].add(i) 
        except:
          # when the set is empty
            nj[word] = {i}

In [0]:
# counting the number of elements present in the set part if nj dictionary. 
# This gives the number of documents in which a word is occurring.
for i in nj:
    nj[i] = len(nj[i])

Now nj contains a dictionary representing a word and the number of documents in which that word is used.
Total number of elements present in nj dictionary should give the entire set of words that are used in our dataset.
 

In [0]:
vocab_size = len(nj)

In [0]:
# creating a list of vocab words of our dataset
total_vocab = [x for x in nj]

In [0]:
# This function accesses the nj dictionary and gives the document count for the word passed into the function
def document_frequency(word):
    counts = 0
    try:
        counts = nj[word]
    except:
        pass
    return counts

<h2>Calculating tf-idf</h2>

In [0]:
doc = 0
tf_idf = {}
# tf_idf is a dictionary that is of the form (document number, token) : tf-idf value
for i in range(N):
    tokens = processed_text[i]
    # A Counter is a container that keeps track of how many times equivalent values are added. The method 'Counter' returns a dictionary
    # with key value as token and value as counts
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = document_frequency(token)
        idf = np.log((N+1)/(df+1))
  
        tf_idf[doc, token] = tf*idf

    doc += 1

In [0]:
tf_idf

<h2>Finding similarity</h2>



In [0]:
# function to calculate the cosine similarity 
def cosine_sim(x,y):
    similarity = np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return similarity

In [0]:
#  'tf_idf_matrix' is a matrix initialized to zeros. The matrix tf_idf_matrix gives a tabular represenation of tf_idf values of all vocab words(columns)
#  arranged according to the documents to which they belong to. 
# Each row of tf_idf_matrix represents a document and each column of tf_idf_matrix represents a word of the vocab. 

tf_idf_matrix = np.zeros((N, vocab_size))
for i in tf_idf:
    try:
        # extract the token from the key value of tf_idf dictionary and put it into the variable cell.
        cell = total_vocab.index(i[1])
        # Inserting the calculated tf_idf value of the token into a position in the D matrix.
        tf_idf_matrix[i[0]][cell] = tf_idf[i]
    except:
        pass

In [0]:
tf_idf_matrix

In [0]:
# Function for performing tf-idf calculations on the query document. Same as that of tf_idf code that's written above. 
def query_document_vector(tokens):
    query_tf_idf_matrix = np.zeros((len(total_vocab)))
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = document_frequency(token)
        idf = math.log((N+1)/(df+1))

        try:
            cell = total_vocab.index(token)
            query_tf_idf_matrix[cell] = tf*idf
        except:
            pass
    return query_tf_idf_matrix

In [0]:
# Function that generates a list of cosine similarity values for the query
def cosine_similarity(query):
    preprocessed_query_document = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query_document))
    
    print("\nGiven Query:", query)
    print("")
    
    # A list for storing the cosine similarities generated between documents
    cosine_values = []
    # creating tf_idf vector for query document
    query_td_idf_vector = query_document_vector(tokens)
    
    for xi in tf_idf_matrix:
      # Calculating cosine similarities between (d1, d_query) to (d4, d_query)
        cosine_values.append(cosine_sim(query_td_idf_vector, xi))
      
    # Sorting the 'cosine_values' list and returning it in the form of corresponding sorted indices using 'argsort()' function.
    output = np.array(cosine_values).argsort()[::-1]
    # 'output' is a list that contains the document numbers sorted(Descending) according to their cosine values.
    #  
    return output

Instead of using the query document as a txt file I've used the contents of d_query.txt in my 'cosine_similarity' function

In [38]:
sorted_similarites = cosine_similarity("java coffee mocha")


Given Query: java coffee mocha



In [39]:
# Printing the document with maximum cosine similarity
print_document(sorted_similarites[0])

d4.txt
Java coffee refers to coffee beans produced in the Indonesian island of Java.  The Indonesian phrase Kopi Java refers not only to the origin of the coffee, but is used to distinguish a style of strong, black, and very sweet coffee.  In some countries, Java can refer to coffee in general.

The coffee is primarily grown on large estates that were built by the Dutch in the 18th century. The five largest estates are Blawan (also spelled Belawan or Blauan), Jampit (or Djampit), Pancoer (or Pancur), Kayumas and Tugosari, and they cover more than 4,000 hectares.

These estates transport ripe cherries quickly to their mills after harvest. The pulp is then fermented and washed off, using the wet process, with rigorous quality control. This results in coffee with good, heavy body and a sweet overall impression. They are sometimes rustic in their flavor profiles, but display a lasting finish. At their best, they are smooth and supple and sometimes have a subtle herbaceous note in the after